# Chapter 5 code for Julia for Machine Learning book

(c) 2020 by Zacharias Voulgaris, all rights reserved.

This code has been tested for Julia 1.4.1. 

# Initialization

## Packages

Get these packages in memory for the experiments that follow.

In [12]:
using CSV, DataFrames;

In [13]:
using Clustering, DecisionTree, GLM, XGBoost;

In [14]:
using StatsBase, Distributions, HypothesisTests, MultivariateStats;

In [15]:
using Distances, MLLabelUtils, MLBase;

In [16]:
using TSne, Gadfly;

In [17]:
using ScikitLearn.CrossValidation: cross_val_score;

In [18]:
using Random;

Note: if you got an alarming large area of red in the outputs of these commands, don't panic. Most of these are just warnings that most likely won't affect the experience of using these packages. Also, the first time you run these packages it is bound to take more time than other times.
 

## Auxiliary functions

In [19]:
function partition(ind::Union{Array{Int64, 1}, UnitRange{Int64}}, r::Float64, shuffle::Bool = true)
    if typeof(ind) == UnitRange{Int64}; ind = collect(ind); end
    N = length(ind) # total number of data points in sample
    n = round(Int64, N*r) # number of data points in training set (train)
    train = [0, 0] # initialize train output
    test = [0, 0] # initialize test output
    
    if shuffle        
        ind_ = ind[randperm(N)]
    else
        ind_ = ind
    end
    
    train = ind_[1:n]
    test = ind_[(n+1):end]
    return train, test
end

partition (generic function with 2 methods)

# Load data from localization.csv file

In [20]:
df = CSV.read("localization.csv", header = false);

In [21]:
old_names = names(df)

8-element Array{String,1}:
 "Column1"
 "Column2"
 "Column3"
 "Column4"
 "Column5"
 "Column6"
 "Column7"
 "Column8"

In [12]:
new_names = map(Symbol, ["WiFi1", "WiFi2", "WiFi3", "WiFi4", "WiFi5", "WiFi6", "WiFi7", "Room"])

8-element Array{Symbol,1}:
 :WiFi1
 :WiFi2
 :WiFi3
 :WiFi4
 :WiFi5
 :WiFi6
 :WiFi7
 :Room

In [13]:
new_names

8-element Array{Symbol,1}:
 :WiFi1
 :WiFi2
 :WiFi3
 :WiFi4
 :WiFi5
 :WiFi6
 :WiFi7
 :Room

In [14]:
for i = 1:8
    rename!(df, old_names[i] => new_names[i])
end

In [15]:
names(df)

8-element Array{String,1}:
 "WiFi1"
 "WiFi2"
 "WiFi3"
 "WiFi4"
 "WiFi5"
 "WiFi6"
 "WiFi7"
 "Room"

In [16]:
df[:RegressionTarget] = Matrix(df[[1, 4, 6, 7]]) * [5, 10, 15, 20] + 0.01*randn(2000)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[16]:1
└ @ Core In[16]:1
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[16]:1
└ @ Core In[16]:1


2000-element Array{Float64,1}:
 -3829.9927919586935
 -3964.996508185597
 -3940.003954577194
 -3875.000908273507
 -3899.991778022533
 -3960.0073734305693
 -3980.0052514356635
 -3910.032825609012
 -3885.0192579638756
 -4099.99940695536
 -4069.99395461761
 -3905.017828659455
 -3905.0179110583836
     ⋮
 -3815.0034319620927
 -3655.009084669348
 -3780.001371980961
 -3720.006566034585
 -3970.0040726666098
 -4010.009619427603
 -3999.992813132378
 -4125.000745279454
 -4020.0021038009913
 -4024.989793418792
 -3969.9981485671083
 -3954.9989804582256

In [17]:
first(df, 10)

,WiFi1,WiFi2,WiFi3,WiFi4,WiFi5,WiFi6,WiFi7,Room,RegressionTarget
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,-64,-56,-61,-66,-71,-82,-81,1,-3829.99
2,-68,-57,-61,-65,-71,-85,-85,1,-3965.0
3,-63,-60,-60,-67,-76,-85,-84,1,-3940.0
4,-61,-60,-68,-62,-77,-90,-80,1,-3875.0
5,-63,-65,-60,-63,-77,-81,-87,1,-3899.99
6,-64,-55,-63,-66,-76,-88,-83,1,-3960.01
7,-65,-61,-65,-67,-69,-87,-84,1,-3980.01
8,-61,-63,-58,-66,-74,-87,-82,1,-3910.03
9,-65,-60,-59,-63,-76,-86,-82,1,-3885.02


# Work the data using StatsBase package

In [11]:
XX = map(Float64, Matrix(df[1:7]));

LoadError: UndefVarError: df not defined

In [10]:
X = StatsBase.standardize(ZScoreTransform, XX, dims=2)
size(X)

LoadError: UndefVarError: XX not defined

In [20]:
df2 = DataFrame(hcat(X, df[:Room]))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[20]:1
└ @ Core In[20]:1


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.478077,1.28936,0.782308,0.275256,-0.231795,-1.34731,-1.2459,1.0
2,0.207303,1.20495,0.84217,0.479389,-0.0647823,-1.33451,-1.33451,1.0
3,0.708444,0.98395,0.98395,0.341103,-0.485416,-1.31193,-1.2201,1.0
4,0.884579,0.971791,0.274095,0.797367,-0.510813,-1.64457,-0.772449,1.0
5,0.739762,0.551459,1.02222,0.739762,-0.578359,-0.954965,-1.51987,1.0
6,0.563639,1.31916,0.647586,0.395747,-0.443716,-1.45107,-1.03134,1.0
7,0.605834,1.00033,0.605834,0.408586,0.211337,-1.5639,-1.26802,1.0
8,0.823905,0.643676,1.09425,0.373332,-0.347585,-1.51908,-1.0685,1.0
9,0.465976,0.919009,1.00962,0.647189,-0.530695,-1.43676,-1.07433,1.0


In [23]:
for i = 1:8
    rename!(df2, names(df2)[i] => new_names[i])
end

In [24]:
first(df2, 10)

,WiFi1,WiFi2,WiFi3,WiFi4,WiFi5,WiFi6,WiFi7,Room
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.478077,1.28936,0.782308,0.275256,-0.231795,-1.34731,-1.2459,1.0
2,0.207303,1.20495,0.84217,0.479389,-0.0647823,-1.33451,-1.33451,1.0
3,0.708444,0.98395,0.98395,0.341103,-0.485416,-1.31193,-1.2201,1.0
4,0.884579,0.971791,0.274095,0.797367,-0.510813,-1.64457,-0.772449,1.0
5,0.739762,0.551459,1.02222,0.739762,-0.578359,-0.954965,-1.51987,1.0
6,0.563639,1.31916,0.647586,0.395747,-0.443716,-1.45107,-1.03134,1.0
7,0.605834,1.00033,0.605834,0.408586,0.211337,-1.5639,-1.26802,1.0
8,0.823905,0.643676,1.09425,0.373332,-0.347585,-1.51908,-1.0685,1.0
9,0.465976,0.919009,1.00962,0.647189,-0.530695,-1.43676,-1.07433,1.0


Note: the localization dataset is used as an example as it's fairly straight-forward and doesn't require pretty much any data engineering. Feel free to experiment with different datasets, for your predictive analytics projects.
 

# Work the data using Clustering package

In [ ]:
nc = 4; # number of clusters

In [30]:
n, m = size(X)

(2000, 7)

In [ ]:
XX = Array{Float64}(undef, m, n);

for i = 1:m
    XX[i,:] = X[:,i]
end

# alternatively, if you use the LinearAlgebra package, you can do the same thing as follows: XX = collect(adjoint(X))

In [ ]:
R = kmeans(XX, nc; maxiter=200, display=:iter) # the second part of the arguments is optional but useful in a demo

In [ ]:
@assert nclusters(R) == nc # although it's pretty obvious from the previous output that the number of clusters is indeed 4, it doesn't hurt to verify it

In [ ]:
a = assignments(R) # assignments (labels) of the various points of the datasets, according to the Kmeans model

In [ ]:
c = counts(R) # Cluster sizes

In [ ]:
M = (R.centers)' # cluster centers (centroids), transposed for easier viewing

In [ ]:
R2 = fuzzy_cmeans(XX, nc, 2, maxiter=200, display=:iter) # 2 is the fuzziness parameter (needs to be > 1)

In [ ]:
M2 = (R2.centers)' # the centroids, transposed for easier viewing

In [ ]:
memberships = R2.weights # how much each data point belongs to each one of the (4) clusters according to Cmeans

# Work the data using predictive ML models

## Prepare data for ML models

In [31]:
X = df2[1:7];

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[31]:1
└ @ Core In[31]:1


In [32]:
XX = map(Float64, Matrix(X));

In [33]:
y1 = map(string, df[:Room]);

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[33]:1
└ @ Core In[33]:1


In [34]:
y2 = df[end];

┌ Warning: `lastindex(df::AbstractDataFrame)` is deprecated, use `ncol(df)` instead.
│   caller = top-level scope at In[34]:1
└ @ Core In[34]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[34]:1
└ @ Core In[34]:1


In [35]:
train, test = partition(1:n, 0.7, true) # 70-30 split of the data

([701, 193, 1971, 123, 878, 1423, 1603, 1484, 621, 1253  …  206, 830, 422, 1923, 1164, 1262, 1826, 1528, 262, 638], [790, 1626, 547, 1748, 187, 46, 1444, 171, 1200, 557  …  1941, 109, 1727, 1666, 1434, 446, 1803, 915, 1675, 1910])

## Build and run the ML models

### Decision Tree

In [36]:
tree = DecisionTreeClassifier(max_depth=3);

In [37]:
DecisionTree.fit!(tree, XX[train,:], y1[train])

DecisionTreeClassifier
max_depth:                3
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  ["1", "2", "3", "4"]
root:                     Decision Tree
Leaves: 7
Depth:  3

In [38]:
print_tree(tree)

Feature 5, Threshold 0.4224564984399155
L-> Feature 2, Threshold 0.7768935529858114
    L-> Feature 5, Threshold -0.30513023260694877
        L-> 2 : 316/369
        R-> 3 : 278/326
    R-> Feature 2, Threshold 1.0265212480612247
        L-> 1 : 107/128
        R-> 1 : 220/220
R-> Feature 3, Threshold 0.4511006717139975
    L-> Feature 4, Threshold 0.4329713570117495
        L-> 4 : 5/5
        R-> 1 : 2/3
    R-> 4 : 349/349


In [39]:
yhat = DecisionTree.predict(tree, XX[test,:])

600-element Array{String,1}:
 "3"
 "4"
 "2"
 "4"
 "1"
 "1"
 "3"
 "1"
 "1"
 "2"
 "3"
 "4"
 "1"
 ⋮
 "1"
 "3"
 "4"
 "1"
 "4"
 "4"
 "3"
 "1"
 "4"
 "2"
 "4"
 "4"

In [40]:
sum(yhat .== y1[test]) / length(test)

0.9233333333333333

In [41]:
accuracy = cross_val_score(tree, XX, y1, cv=5)

5-element Array{Float64,1}:
 0.8075
 0.9075
 0.8975
 0.89
 0.87

### Random Forests

In [ ]:
n_subfeatures = 3;
n_trees = 10;
pst = 0.5; # portion of samples per tree
max_depth = 6;
n_folds = 5; # number of folds for the K-fold cross-validation later on

In [ ]:
forest1 = build_forest(y1[train], XX[train,:], n_subfeatures, n_trees, pst, max_depth);

In [ ]:
yhat = apply_forest(forest1, XX[test,:])

In [ ]:
scores = apply_forest_proba(forest1, XX[test,:], map(string, 1:4)) # map(string, 1:4) => ["1", "2", "3", "4"] but it's more elegant and less risky

In [ ]:
accuracy = nfoldCV_forest(y1, XX, n_folds, n_subfeatures)

In [ ]:
forest2 = build_forest(y2[train], XX[train,:], 2, 10, 0.5, 6);

In [ ]:
apply_forest(forest2, XX[test,:])

In [ ]:
accuracy = nfoldCV_forest(y2, XX, n_folds, n_subfeatures)

In [ ]:
ni = 10 # number of iterations for boosted stumps

In [ ]:
model, coeffs = build_adaboost_stumps(y1[train], XX[train,:], ni)

In [ ]:
apply_adaboost_stumps(model, coeffs, XX[test,:])

In [ ]:
apply_adaboost_stumps_proba(model, coeffs, XX[test,:], map(string, 1:4))

In [ ]:
accuracy = nfoldCV_stumps(y1, XX, n_folds, ni)

If this isn't an overfit model, I don't know what is!
 

# ML utilities packages

## MLBase functions

For the love of God, don't bother with the label functions of this package as they are a nightmare to figure out and do something useful with them! Fortunately, there is the MLLabelsUtils package for this (see next section).

### Preprocessing target and prediction variables

In [ ]:
pred = map(x -> parse(Int64, x), yhat); # prediction of RF classifier

In [ ]:
gt = map(x -> parse(Int64, x), y1[test]); # ground truth for classification

In [ ]:
conf = maximum(scores, dims=2)[:]; # confidence scores for RF classifier

In [ ]:
k = length(unique(gt)); # number of classes

### Performance evaluation functions for classification

In [ ]:
correctrate(gt, pred) # accuracy rate

In [ ]:
confusmat(k, gt, pred) # confusion matrix for the RF classifier

In [ ]:
ROC =  roc(gt, conf) # Receiver Operator Characteristic data (for building an ROC curve) for RFC

In [ ]:
r = ROC[50] # a particular instance of the ROC data array

In [ ]:
precision(r)

In [ ]:
recall(r)

In [ ]:
f1score(r)

### Crossvalidation functions

In [ ]:
n = length(y1) # total number of data points
k = round(Int64, 0.3*n) # size of a sample for test set
sn = 5 # number of samples

In [ ]:
Kfold(n, k)

In [ ]:
LOOCV(n)

In [ ]:
RandomSub(n, sn, k)

## MLLabelsUtils functions

In [ ]:
y1[rand(1:n, 20)] # 20 random labels just to remind ourselves of what the classification target variable is like

In [ ]:
labelenc(y1)

In [ ]:
islabelenc(y1, LabelEnc.ZeroOne)

In [ ]:
convertlabel([:Room1,:Room2,:Room3,:Room4], y1)

In [ ]:
convertlabel(LabelEnc.Indices{Int}, y1) 

In [ ]:
convertlabel(LabelEnc.OneOfK{Bool}, y1)

In [ ]:
convertlabel(LabelEnc.OneOfK{Float64}, y1)

In [ ]:
convertlabel(LabelEnc.TrueFalse, y1, LabelEnc.OneVsRest("1"))

In [ ]:
scores[1:10,:] # confidence scores for the RF classifier

In [ ]:
MLLabelUtils.classify(scores[1:10,:]', LabelEnc.OneOfK) # note: the scores are transposed using the ' operator

# Make Use of the "Other" Packages for Some Cool Plots

## Distances package

In [ ]:
D = [Euclidean(), Cityblock(), Jaccard(), CosineDist(), CorrDist()];

In [ ]:
y = map(Float64, y2);

In [ ]:
for dist in D
    println(Distances.evaluate(dist, X[:,1], y2))
end

In [ ]:
CosineDist()(X[:,1], y2)

In [ ]:
D = pairwise(Jaccard(), XX, dims=2)

Note that the pairwise function works only with matrices, hence the use of XX here instead of X (X is a dataframe)

## Gadfly package

In [ ]:
plot(df, x = :WiFi1, y = :RegressionTarget)

Clearly there is a trend here, showing that the RegressionTarget variable is related to the WiFi1 variable

In [ ]:
plot(df2, x = :WiFi1, y = :WiFi2, color = :Room)

In [ ]:
plot(df3, x = :x1, y = :x2, color = :x3)

In [ ]:
plot(df2, x = "WiFi1", Geom.histogram)

In [ ]:
plot(df2, x = "WiFi1", Geom.histogram(bincount=30))

In [ ]:
plot(df2, x=:WiFi1, y=:WiFi2, Geom.histogram2d(xbincount=40, ybincount=40))

In [ ]:
spy(D) # heatmap of the Jaccard distance matrix from a previous section

# Sandbox

A place for you to experiment with the above material, without making a mess of the whole notebook or disrupt its flow. It's much easier to maintain this section than keep track of various copies of this notebook.